In [33]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [34]:
df=pd.read_csv("Surgical-deepnet.csv")
df

,bmi,Age,asa_status,baseline_cancer,baseline_charlson,baseline_cvd,baseline_dementia,baseline_diabetes,baseline_digestive,baseline_osteoart,...,complication_rsi,dow,gender,hour,month,moonphase,mort30,mortality_rsi,race,complication
0,19.31,59.2,1,1,0,0,0,0,0,0,...,-0.57,3,0,7.63,6,1,0,-0.43,1,0
1,18.73,59.1,0,0,0,0,0,0,0,0,...,0.21,0,0,12.93,0,1,0,-0.41,1,0
2,21.85,59.0,0,0,0,0,0,0,0,0,...,0.00,2,0,7.68,5,3,0,0.08,1,0
3,18.49,59.0,1,0,1,0,0,1,1,0,...,-0.65,2,1,7.58,4,3,0,-0.32,1,0
4,19.70,59.0,1,0,0,0,0,0,0,0,...,0.00,0,0,7.88,11,0,0,0.00,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14630,18.79,14.1,1,0,1,0,0,0,0,0,...,-0.54,1,0,7.78,0,1,0,-0.16,1,1
14631,19.65,12.6,0,0,0,0,0,0,0,0,...,-1.42,4,0,8.40,6,1,0,-0.77,1,1
14632,14.84,12.6,1,0,0,0,0,0,0,0,...,0.65,0,0,13.25,3,3,0,0.99,1,1
14633,17.75,8.9,0,0,1,0,0,0,1,0,...,-0.50,0,1,8.30,5,0,0,0.17,1,1


In [35]:
X=df.drop("complication",axis=1).copy().values
Y=df["complication"].copy().values.reshape(-1,1)

In [36]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
Y_train.shape

(11708, 1)

In [37]:
class Perceptron:
    def __init__(self,learning_rate, input_length):
        self.learning_rate=learning_rate
        self.weights=np.random.rand(input_length)
        self.bias=np.random.rand(1)

    def activation(self, x, function):
        if function == 'sigmoid':
            return 1/(1+np.exp(-x))
        elif function == 'relu':
            return np.maximum(0,x)
        elif function == 'tanh':
            return np.tanh(x)
        elif function == 'linear':
            return x
        else:
            raise Exception('Unkown activayion function')

    def fit(self,X_train,Y_train,epochs):
        for epoch in tqdm(range(epochs)):
            for x,y in zip(X_train,Y_train):
                # forwarding
                y_pred = x @ self.weights + self.bias
                y_pred = self.activation(y_pred,'sigmoid')

                # back propagation
                error=y - y_pred

                # updating
                self.weights += self.learning_rate*error*x
                self.bias += self.learning_rate*error


    def predict(self,X_test):
        Y_perd=[]
        for x_test in X_test:
            y_pred= x_test @ self.weights + self.bias
            y_pred= self.activation(y_pred, 'sigmoid')
            Y_perd.append(y_pred)
        return np.array(Y_perd)

    def calculate_loss(self,X_test,Y_test,metric):
        Y_pred = self.predict(X_test)
        if metric == 'mse':
            return np.mean(np.square(Y_test-Y_pred))
        elif metric == 'mae':
            return np.mean(np.abs(Y_test-Y_pred))
        elif metric == 'rmse':
            return np.sqrt(np.mean(np.square(Y_test-Y_pred)))
        else:
            raise Exception('Unknown metric')
        
    def calculate_accuracy(self,X_test,Y_test):
        Y_pred=self.predict(X_test)
        Y_pred=np.where(Y_pred > 0.5, 1, 0)
        accuracy=np.sum(Y_pred==Y_test)/len(Y_pred)
        return accuracy

    def evaluate(self,X_test,Y_test):
        loss = self.calculate_loss(X_test,Y_test,'mse')
        accuracy = self.calculate_accuracy(X_test,Y_test)
        return loss,accuracy

In [38]:
model=Perceptron(learning_rate=0.001,input_length=X_train.shape[1])
model.fit(X_train,Y_train,epochs=256)

100%|██████████| 256/256 [01:14<00:00,  3.45it/s]


In [39]:
print('loss, accuracy= ')
model.evaluate(X_test,Y_test)

loss, accuracy= 


(0.14386783085241117, 0.8008199521694568)

In [40]:
Y_pred=model.predict(X_test)
Y_pred=np.where(Y_pred > 0.5, 1, 0)

In [41]:
from sklearn.metrics import confusion_matrix

confusion_matrix(Y_test,Y_pred)

array([[2005,  193],
       [ 390,  339]], dtype=int64)